In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import keras
from keras.models import load_model
from keras import backend as K
from keras import models
from keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from keras_tuner.tuners import RandomSearch
import keras_tuner

In [2]:
from dataset import load_dataset, load_labels, convert_to_epochs
from features import time_series_features, nonlinear_features, entropy_features, hjorth_features, freq_band_features

In [3]:
num_classes = 2
channels = 32
sfreq = 128

In [4]:
dataset_ = load_dataset(raw = False)
dataset = convert_to_epochs(dataset_, channels, sfreq)

In [5]:
features = time_series_features(dataset, channels)
# freq_bands = np.array([1, 4, 8, 13, 31, 50])
# features = freq_band_features(dataset, channels, sfreq, freq_bands)
# features = nonlinear_features(dataset, channels)
# features = hjorth_features(dataset, channels, sfreq)
# features = entropy_features(dataset, channels, sfreq)
data = features

In [6]:
labels = load_labels()
label = pd.concat([labels['t1_math'], labels['t2_math'],
                  labels['t3_math']]).to_numpy()
label = label.repeat(dataset.shape[1])

# Linear Regression

In [7]:
x_train, x_test, y_train, y_test = train_test_split(
    data, label, test_size=0.33, random_state=42)
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
lr_clf = LogisticRegression(max_iter=1000).fit(x_train, y_train)
y_pred = lr_clf.predict(x_test)
y_true = y_test


In [8]:
scores_lr = lr_clf.score(x_test, y_test)
precision_lr = metrics.precision_score(y_true, y_pred, average='macro')
recall_lr = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_lr = metrics.f1_score(y_true, y_pred, average='weighted')
print('accuracy is:', scores_lr)
print('precision is:', precision_lr)
print('recall is:', recall_lr)
print('f1_score is:', f1_score_lr)
metrics.confusion_matrix(y_true, y_pred)


accuracy is: 0.6646464646464646
precision is: 0.6633407167510779
recall is: 0.6646464646464646
f1_score is: 0.6647288864063057


array([[359, 168],
       [164, 299]])

# KNN Classifier

In [9]:
K.clear_session()
x_train, x_test, y_train, y_test = train_test_split(
    data, label, test_size=0.33, random_state=42)
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
neigh_clf = KNeighborsClassifier(n_neighbors=2)
neigh_clf.fit(x_train, y_train)
y_pred = neigh_clf.predict(x_test)
y_true = y_test


In [10]:
scores_neigh = neigh_clf.score(x_test, y_test)
precision_neigh = metrics.precision_score(y_true, y_pred, average='macro')
recall_neigh = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_neigh = metrics.f1_score(y_true, y_pred, average='weighted')
print('accuracy is:', scores_neigh)
print('precision is:', precision_neigh)
print('recall is:', recall_neigh)
print('f1_score is:', f1_score_neigh)
metrics.confusion_matrix(y_true, y_pred)


accuracy is: 0.6323232323232323
precision is: 0.642935839115468
recall is: 0.6323232323232323
f1_score is: 0.6165170220725775


array([[430,  97],
       [267, 196]])

# SVM Classifier

In [11]:
x_train, x_test, y_train, y_test = train_test_split(
data, label, test_size=0.33, random_state=42)
svm_clf = SVC(C=10, kernel='rbf')
svm_clf.fit(x_train, y_train)
y_pred = svm_clf.predict(x_test)
y_pred_train = svm_clf.predict(x_train)
y_true = y_test


In [12]:
scores_svm_train = svm_clf.score(x_train, y_train)
precision_svm_train = metrics.precision_score(y_train, y_pred_train, average='macro')
recall_svm_train = metrics.recall_score(y_train, y_pred_train, average='micro')
f1_score_svm_train = metrics.f1_score(y_train, y_pred_train, average='weighted')
print('accuracy is:', scores_svm_train)
print('precision is:', precision_svm_train)
print('recall is:', precision_svm_train)
print('f1_score is:', f1_score_svm_train)
# metrics.confusion_matrix(y_true, y_pred)


accuracy is: 0.8084577114427861
precision is: 0.8083113276835878
recall is: 0.8083113276835878
f1_score is: 0.8083628691115632


In [13]:
scores_svm = svm_clf.score(x_test, y_test)
precision_svm = metrics.precision_score(y_true, y_pred, average='macro')
recall_svm = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_svm = metrics.f1_score(y_true, y_pred, average='weighted')
print('accuracy is:', scores_svm)
print('precision is:', precision_svm)
print('recall is:', recall_svm)
print('f1_score is:', f1_score_svm)
metrics.confusion_matrix(y_true, y_pred)


accuracy is: 0.7575757575757576
precision is: 0.7569322750231935
recall is: 0.7575757575757576
f1_score is: 0.7572081522702866


array([[416, 111],
       [129, 334]])

# Neural Network

In [14]:
K.clear_session()
y_v = label
y_v = to_categorical(y_v, num_classes)
x_train, x_test, y_train, y_test = train_test_split(data, y_v, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=1)

In [15]:
def model_builder(hp):
    model = keras.models.Sequential()
    model.add(keras.Input(shape=(x_train.shape[1],)))

    for i in range(hp.Int('layers', 2, 6)):
      model.add(keras.layers.Dense(units=hp.Int('units_' + str(i), 32, 1024, step=32),
                                    activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid'])))

    model.add(keras.layers.Dense(num_classes, activation='softmax', name='out'))
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.adam_v2.Adam(learning_rate=hp_learning_rate),
                loss = "binary_crossentropy",
                metrics=['accuracy'])
    return model

In [16]:
tuner = RandomSearch(
    model_builder,
    objective = 'val_accuracy',
    max_trials = 5,
    executions_per_trial = 2,
    overwrite=True
)

Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2022-11-02 10:05:26.383610: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-02 10:05:26.383742: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [17]:
tuner.search_space_summary()

Search space summary
Default search space size: 6
layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 1024, 'step': 32, 'sampling': None}
act_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 1024, 'step': 32, 'sampling': None}
act_1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [18]:
tuner.search(x_train, y_train, epochs = 50, validation_data= [x_val, y_val])

Trial 5 Complete [00h 01m 12s]
val_accuracy: 0.5658333599567413

Best val_accuracy So Far: 0.9291666746139526
Total elapsed time: 00h 05m 19s
INFO:tensorflow:Oracle triggered exit


In [19]:
model = tuner.get_best_models(num_models=1)[0]

In [20]:
y_pred = model.predict(x_test)
y_true = y_test
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_true, axis=1)

2022-11-02 10:10:46.007728: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [21]:
scores_dnn = model.evaluate(x_test, y_test, verbose=0)
precision_dnn = metrics.precision_score(y_true, y_pred, average='macro')
recall_dnn = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_dnn = metrics.f1_score(y_true, y_pred, average='weighted')
print('accuracy is:', scores_dnn[1])
print('precision is:', precision_dnn)
print('recall is:', recall_dnn)
print('f1_score is:', f1_score_dnn)
metrics.confusion_matrix(y_true, y_pred)

2022-11-02 10:10:46.205803: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


accuracy is: 0.9283333420753479
precision is: 0.9287769302823499
recall is: 0.9283333333333333
f1_score is: 0.9282928266682259


array([[293,  18],
       [ 25, 264]])